# Practical Usage of Lava w/ custom input from the lab

The input data from the lab used in this example consists of a sequence of timestamps where a specific neuron fired. The data is stored in a csv file, where each value is a timestamp (in milliseconds). 

Thus, the input layer only requires 1 connection at each timestep. We will assume the network contains 5 neurons for the sake of this example.

Let's start by defining the middle layer, the 2-layered feed-forward network of LIF neurons

In [2]:
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense

LIF?

Init signature: LIF(*args, **kwargs)
Docstring:     
Leaky-Integrate-and-Fire (LIF) neural Process.

LIF dynamics abstracts to:
u[t] = u[t-1] * (1-du) + a_in         # neuron current
v[t] = v[t-1] * (1-dv) + u[t] + bias  # neuron voltage
s_out = v[t] > vth                    # spike if threshold is exceeded
v[t] = 0                              # reset at spike

Parameters
----------
shape : tuple(int)
    Number and topology of LIF neurons.
u : float, list, numpy.ndarray, optional
    Initial value of the neurons' current.
v : float, list, numpy.ndarray, optional
    Initial value of the neurons' voltage (membrane potential).
du : float, optional
    Inverse of decay time-constant for current decay. Currently, only a
    single decay can be set for the entire population of neurons.
dv : float, optional
    Inverse of decay time-constant for voltage decay. Currently, only a
    single decay can be set for the entire population of neurons.
bias_mant : float, list, numpy.ndarray, optiona

In [5]:
import numpy as np

# Create Processes
lif1 = LIF(shape=(5,),  # There are 5 neurons
           vth=10,
           dv=0.1,
           du=0.1,
           bias_mant=0,
           name="lif1")

dense = Dense(weights=np.random.rand(2, 5),     # 5 nodes in the input side, 2 nodes on the output side 
              name="dense1") 

lif2 = LIF(shape=(2,),
           vth=10,
           dv=0.1,
           du=0.1,
           bias_mant=0,
           name="lif2")

Should the Dense layer contain 2 output nodes? I was thinking each output node could correspond to the probability of the events "network burst detected" or "network burst not detected" at each timestep. It might be better to represent it with a single node?

Let's take a look at the ports and connections of the middle layer:

In [6]:
for proc in [lif1, dense, lif2]:
    for port in proc.in_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
    for port in proc.out_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")

Proc: lif1  Port Name: a_in  Size: 5
Proc: lif1  Port Name: s_out Size: 5
Proc: dense1 Port Name: s_in  Size: 5
Proc: dense1 Port Name: a_out Size: 2
Proc: lif2  Port Name: a_in  Size: 2
Proc: lif2  Port Name: s_out Size: 2
